# Graph-based Adversarial Machine Learning

# Import Libraries

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torch-geometric pywavelets

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch.optim import Adam
import numpy as np

# Load Dataset

In [ ]:
# Load the dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]
print(data.x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


# Define Graph Neural Network

In [ ]:
# Define the GNN
class GNN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Set Hyperparameters

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train Model

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Get the Accuracy of the Model

In [ ]:
# Evaluate the model
def evaluate(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
    accuracy = correct / data.test_mask.sum().item()
    return accuracy

accuracy_before = evaluate(model, data)
print(f'Accuracy before attacks: {accuracy_before:.4f}')

Accuracy before attacks: 0.8150


# FGSM Attack

In [ ]:
# FGSM Attack
def fgsm_attack(model, data, epsilon):
    data.x.requires_grad = True
    output = model(data)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    model.zero_grad()
    loss.backward()
    data_grad = data.x.grad.data
    sign_data_grad = data_grad.sign()
    perturbed_data = data.x + epsilon * sign_data_grad
    perturbed_data = torch.clamp(perturbed_data, 0, 1)
    return perturbed_data

epsilon = 0.1
data_fgsm = data.clone()
data_fgsm.x = fgsm_attack(model, data, epsilon)
accuracy_after_fgsm = evaluate(model, data_fgsm)
print(f'Accuracy after FGSM attack: {accuracy_after_fgsm:.4f}')

Accuracy after FGSM attack: 0.1150


# PGD Attack

In [ ]:
def pgd_attack(model, data, epsilon, alpha, num_iter):
    perturbed_data = data.x.clone().detach().requires_grad_(True)
    for _ in range(num_iter):
        data.x = perturbed_data  # Use perturbed_data in the forward pass
        output = model(data)
        loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
        model.zero_grad()
        loss.backward()
        data_grad = perturbed_data.grad.data
        perturbed_data = perturbed_data + alpha * data_grad.sign()
        perturbation = torch.clamp(perturbed_data - data.x, -epsilon, epsilon)
        perturbed_data = torch.clamp(data.x + perturbation, 0, 1)
        perturbed_data = perturbed_data.detach().requires_grad_(True)
    return perturbed_data

alpha = 0.01
num_iter = 40
data_pgd = data.clone()
data_pgd.x = pgd_attack(model, data, epsilon, alpha, num_iter)
accuracy_after_pgd = evaluate(model, data_pgd)
print(f'Accuracy after PGD attack: {accuracy_after_pgd:.4f}')

Accuracy after PGD attack: 0.0900


# Carlini & Wagner (C&W) Attack

In [ ]:
# Carlini & Wagner (C&W) Attack
def cw_attack(model, data, c=1e-4, lr=0.01, num_iter=1000):
    data_adv = data.clone()
    delta = torch.zeros_like(data.x, requires_grad=True).to(device)
    optimizer = Adam([delta], lr=lr)

    for _ in range(num_iter):
        optimizer.zero_grad()
        adv_data = data.x + delta
        output = model(data)
        loss1 = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
        loss2 = c * torch.norm(delta, p=2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        delta.data = torch.clamp(data.x + delta.data, 0, 1) - data.x
    data_adv.x = torch.clamp(data.x + delta.data, 0, 1)
    return data_adv.x

data_cw = data.clone()
data_cw.x = cw_attack(model, data)
accuracy_after_cw = evaluate(model, data_cw)
print(f'Accuracy after C&W attack: {accuracy_after_cw:.4f}')



Accuracy after C&W attack: 0.0900


# DeepFool Attack

In [ ]:
def deepfool_attack(model, data, num_iter=50, overshoot=0.02):
    data_adv = data.clone()
    perturbed_data = data.x.clone().detach().requires_grad_(True)

    for _ in range(num_iter):
        data.x = perturbed_data  # Use perturbed_data in the forward pass
        output = model(data)
        loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
        model.zero_grad()
        loss.backward()

        # Ensure gradients are not None
        grad = perturbed_data.grad
        if grad is None:
            break

        with torch.no_grad():
            output = model(data)
            _, pred = output.max(dim=1)
            if pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() == 0:
                break
            r_i = overshoot * grad / torch.norm(grad, p=2)
            perturbed_data = perturbed_data + r_i
            perturbed_data = torch.clamp(perturbed_data, 0, 1).detach().requires_grad_(True)

        data.x = perturbed_data  # Update data.x with the new perturbed data

    data_adv.x = perturbed_data.detach()
    return data_adv.x

data_deepfool = data.clone()
data_deepfool.x = deepfool_attack(model, data)
accuracy_after_deepfool = evaluate(model, data_deepfool)
print(f'Accuracy after DeepFool attack: {accuracy_after_deepfool:.4f}')

Accuracy after DeepFool attack: 0.0900


# Train ML Model to Handle Attacks

In [ ]:
model_clean = GNN(dataset.num_node_features, dataset.num_classes).to(device)
model_perturbed = GNN(dataset.num_node_features, dataset.num_classes).to(device)
optimizer_clean = Adam(model_clean.parameters(), lr=0.01, weight_decay=5e-4)
optimizer_perturbed = Adam(model_perturbed.parameters(), lr=0.01, weight_decay=5e-4)

# Perturb the data using FGSM
epsilon = 0.1
perturbed_data = data.clone()
perturbed_data.x = fgsm_attack(model_clean, data, epsilon)

# Training function
def train_mixed(model, optimizer, data, perturbed_data, alpha=0.9):
    model.train()
    optimizer.zero_grad()

    # Clean data loss
    out_clean = model(data)
    loss_clean = F.nll_loss(out_clean[data.train_mask], data.y[data.train_mask])

    # Perturbed data loss
    out_perturbed = model(perturbed_data)
    loss_perturbed = F.nll_loss(out_perturbed[perturbed_data.train_mask], perturbed_data.y[perturbed_data.train_mask])

    # Combined loss
    loss = alpha * loss_clean + (1 - alpha) * loss_perturbed
    loss.backward(retain_graph=True)
    optimizer.step()

# Testing function
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Train model on mixed clean and perturbed data
for epoch in range(200):
    train_mixed(model_perturbed, optimizer_perturbed, data, perturbed_data, alpha=0.9)
    acc_clean = test(model_perturbed, data)
    acc_perturbed = test(model_perturbed, perturbed_data)
    print(f'[Model Perturbed] Epoch {epoch + 1}, Accuracy on clean data: {acc_clean:.4f}, Accuracy on perturbed data: {acc_perturbed:.4f}')

# Final evaluation
clean_acc_perturbed = test(model_perturbed, data)
perturbed_acc_perturbed = test(model_perturbed, perturbed_data)

print(f'Final Accuracy on clean data: {clean_acc_perturbed:.4f}')
print(f'Final Accuracy on perturbed data: {perturbed_acc_perturbed:.4f}')


[Model Perturbed] Epoch 1, Accuracy on clean data: 0.3090, Accuracy on perturbed data: 0.2250
[Model Perturbed] Epoch 2, Accuracy on clean data: 0.3080, Accuracy on perturbed data: 0.3520
[Model Perturbed] Epoch 3, Accuracy on clean data: 0.4250, Accuracy on perturbed data: 0.4410
[Model Perturbed] Epoch 4, Accuracy on clean data: 0.5180, Accuracy on perturbed data: 0.5250
[Model Perturbed] Epoch 5, Accuracy on clean data: 0.5390, Accuracy on perturbed data: 0.5350
[Model Perturbed] Epoch 6, Accuracy on clean data: 0.5580, Accuracy on perturbed data: 0.5470
[Model Perturbed] Epoch 7, Accuracy on clean data: 0.6150, Accuracy on perturbed data: 0.5730
[Model Perturbed] Epoch 8, Accuracy on clean data: 0.6330, Accuracy on perturbed data: 0.5710
[Model Perturbed] Epoch 9, Accuracy on clean data: 0.6280, Accuracy on perturbed data: 0.5580
[Model Perturbed] Epoch 10, Accuracy on clean data: 0.6340, Accuracy on perturbed data: 0.5680
[Model Perturbed] Epoch 11, Accuracy on clean data: 0.6350,